In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/멀티캠퍼스

/content/drive/MyDrive/멀티캠퍼스


In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import shutil
from keras.preprocessing import image
import tensorflow as tf


train = pd.read_csv('./shopee-product-matching/train.csv')
test = pd.read_csv('./shopee-product-matching/test.csv')
sample = pd.read_csv('./shopee-product-matching/sample_submission.csv')

In [4]:
train=train.drop(['image_phash','title'], axis=1)
train.head()

,posting_id,image,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,3648931069


In [5]:
class_img=train['label_group'].drop_duplicates( keep='first', inplace=False)
len(class_img)

11014

In [10]:
for fname in tqdm(class_img):
    if not os.path.exists(f"./kaggle/class/{fname}"):
         os.makedirs(f"./kaggle/class/{fname}")

100%|██████████| 11014/11014 [00:24<00:00, 456.24it/s]


In [6]:
class_file=os.listdir('./kaggle/class/') 
print(len(class_file)) 

11014


In [ ]:
# 기존 방법은 에러.. 해결못함 -> 아래 방법으로 클래스별로 이미지 나누는 작업 
# 너무 느려서 로켤에서 수행
image_path='./shopee-product-matching/train_images'
class_path='./kaggle/class'

for i in tqdm(range(0,len(train['label_group']))):
    shutil.copy(image_path+'/'+ train['image'][i],class_path+'/'+str(train['label_group'][i]))

In [4]:
import json
import math
from tqdm import tqdm 

import cv2
from keras import layers, optimizers
from keras.applications import MobileNetV2
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers, optimizers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
# from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import imagenet_utils
from keras.layers import GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image

In [5]:
generator = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,  # randomly flip images
    preprocessing_function=tf.keras.applications.resnet.preprocess_input
    )

In [6]:
testgenerator = ImageDataGenerator(rescale = 1./255)

In [7]:
target_size = [224, 224]
batch_size = 32

In [8]:
# train_dir= './shopee-product-matching/train_images/'
train_dir= './kaggle/class/'
test_dir='./shopee-product-matching/test_images'

In [13]:
train_gen = generator.flow_from_directory(train_dir,
 target_size = target_size,
 batch_size = batch_size,
 shuffle=True,
 class_mode = 'categorical',                                        
 subset='training')

val_gen = generator.flow_from_directory(train_dir,
 target_size = target_size,
 batch_size = batch_size,
 shuffle=True,
 class_mode = 'categorical',                                        
 subset='validation')

Found 30865 images belonging to 11014 classes.
Found 1596 images belonging to 11014 classes.


In [9]:
base_model=MobileNet(weights='imagenet',include_top=False,input_shape=(224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(224,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(224,activation='relu')(x) #dense layer 2
x=Dense(128,activation='relu')(x) #dense layer 3
preds=Dense(11014,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

17227776/17225924 [==============================] - 0s 0us/step


In [10]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['AUC'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [14]:
model.fit_generator(generator=train_gen,
                    epochs=20,
                    verbose=1,
                    validation_data=val_gen)
model.save('image_model_0423.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
965/965 [==============================] - 11109s 11s/step - loss: 9.3138 - auc: 0.5001 - val_loss: 8.3903 - val_auc: 0.5215
Epoch 2/20
965/965 [==============================] - 557s 577ms/step - loss: 9.0334 - auc: 0.5022 - val_loss: 7.8803 - val_auc: 0.5171
Epoch 3/20
965/965 [==============================] - 551s 571ms/step - loss: 8.6972 - auc: 0.5041 - val_loss: 7.5029 - val_auc: 0.5218
Epoch 4/20
965/965 [==============================] - 557s 577ms/step - loss: 8.4332 - auc: 0.5052 - val_loss: 7.2968 - val_auc: 0.5296
Epoch 5/20
965/965 [==============================] - 548s 568ms/step - loss: 8.2159 - auc: 0.5066 - val_loss: 7.1264 - val_auc: 0.5349
Epoch 6/20
965/965 [==============================] - 544s 563ms/step - loss: 8.0165 - auc: 0.5094 - val_loss: 7.0727 - val_auc: 0.5486
Epoch 7/20
965/965 [==============================] - 547s 567ms/step - loss: 7.8273 - auc: 0.5114 - val_loss: 7.0242 - val_auc: 0.5493
Epoch 8/20
965/965 [============================

In [11]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

#     if show:
#         plt.imshow(img_tensor[0])                           
#         plt.axis('off')
#         plt.show()

    return img_tensor

    

In [17]:
img_path = './shopee-product-matching/test_images'

#tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
posting_id=test['posting_id'].tolist()
# print(sample_sub)

labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
preds=[]

model=tf.keras.models.load_model('image_model_0423.h5')
for test_image in test['image']:
        #predicted_class_indices=[]
        testimg=load_image(os.path.join(img_path,test_image))
        preds.append(model.predict(testimg))
label_group=[]        
for pred in preds:
    predicted_class_indices=np.argmax(pred,axis=1)
    predictions = [labels[k] for k in predicted_class_indices]    
    label_group.append(predictions)    
print(label_group)

sample_sub = pd.DataFrame(posting_id, columns=['posting_id'])
sample_sub = pd.concat([sample_sub, pd.DataFrame(label_group,columns=['label_group'])],axis=1)

# sample_sub['matches']=sample_sub.label_group.map(tmp)
sample_sub.head()

[['731330024'], ['2602613399'], ['3185044263']]


,posting_id,label_group
0,test_2255846744,731330024
1,test_3588702337,2602613399
2,test_4015706929,3185044263


In [18]:
sample_sub['label_group']=pd.DataFrame(sample_sub['label_group'],dtype=np.int)

In [19]:
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
sample_sub['matches']=sample_sub.label_group.map(tmp)

sample_sub.head()

,posting_id,label_group,matches
0,test_2255846744,731330024,"[train_925544261, train_3855533455, train_1824..."
1,test_3588702337,2602613399,"[train_718434073, train_1365034763, train_1764..."
2,test_4015706929,3185044263,"[train_2179692783, train_3046259149, train_391..."
